In [53]:
import re
from dotenv import load_dotenv
import pandas as pd
import json
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from jinja2 import Template
import subprocess
from lib.sql_code_parser import SqlCodeParser

load_dotenv()


True

In [15]:
source_insights_dataframe = pd.read_csv('./results/parsed_code.csv')
source_insights_dataframe.head()

,db_object_name,ddl_operation,sql_code
0,Employee Sales by Country,DROP PROCEDURE,"/*\n** Copyright Microsoft, Inc. 1994 - 2000\n..."
1,Sales by Year,DROP PROCEDURE,"/*\n** Copyright Microsoft, Inc. 1994 - 2000\n..."
2,Ten Most Expensive Products,DROP PROCEDURE,"/*\n** Copyright Microsoft, Inc. 1994 - 2000\n..."
3,CustOrderHist,DROP PROCEDURE,"/*\n** Copyright Microsoft, Inc. 1994 - 2000\n..."
4,CustOrdersDetail,DROP PROCEDURE,"/*\n** Copyright Microsoft, Inc. 1994 - 2000\n..."


In [16]:
# Find all procedures
procedures = source_insights_dataframe[source_insights_dataframe['ddl_operation'] == 'CREATE PROCEDURE']
procedure_names = procedures['db_object_name'].to_list()
procedure_names

['Ten Most Expensive Products',
 'Employee Sales by Country',
 'Sales by Year',
 'CustOrdersDetail',
 'CustOrdersOrders',
 'CustOrderHist',
 'SalesByCategory',
 'InsertProduct',
 'UpdateProduct',
 'DeleteProduct',
 'InsertOrder',
 'UpdateOrder',
 'DeleteOrder',
 'InsertOrderDetails',
 'UpdateOrderDetails',
 'DeleteOrderDetails',
 'ExecuteInsertProduct',
 'byroyalty',
 'reptq1',
 'reptq2',
 'reptq3']

In [17]:
# Find all tables
tables = source_insights_dataframe[source_insights_dataframe['ddl_operation'] == 'CREATE TABLE']
table_names = tables['db_object_name'].to_list()
table_names

['Employees',
 'Categories',
 'Customers',
 'Shippers',
 'Suppliers',
 'Orders',
 'Products',
 'Order Details',
 'CustomerCustomerDemo',
 'CustomerDemographics',
 'Region',
 'Territories',
 'EmployeeTerritories',
 'authors',
 'publishers',
 'titles',
 'titleauthor',
 'stores',
 'sales',
 'roysched',
 'discounts',
 'jobs',
 'pub_info',
 'employee']

In [67]:
# Let's assume this is your code
code = """

 GO

CREATE PROCEDURE CustOrdersDetail @OrderID int
AS
SELECT ProductName,
    UnitPrice=ROUND(Od.UnitPrice, 2),
    Quantity,
    Discount=CONVERT(int, Discount * 100), 
    ExtendedPrice=ROUND(CONVERT(money, Quantity * (1 - Discount) * Od.UnitPrice), 2)
FROM Products P, [Order Details] Od
WHERE Od.ProductID = P.ProductID and Od.OrderID = @OrderID
go


if exists (select * from sysobjects where id = object_id('dbo.CustOrdersOrders'))
	drop procedure dbo.CustOrdersOrders
GO

CREATE PROCEDURE CustOrdersOrders @CustomerID nchar(5)
AS
SELECT OrderID, 
	OrderDate,
	RequiredDate,
	ShippedDate
FROM Orders
WHERE CustomerID = @CustomerID
ORDER BY OrderID
GO


if exists (select * from sysobjects where id = object_id('dbo.CustOrderHist') and sysstat & 0xf = 4)
	drop procedure dbo.CustOrderHist
GO
CREATE PROCEDURE CustOrderHist @CustomerID nchar(5)
AS
SELECT ProductName, Total=SUM(Quantity)
FROM Products P, [Order Details] OD, Orders O, Customers C
WHERE C.CustomerID = @CustomerID
AND C.CustomerID = O.CustomerID AND O.OrderID = OD.OrderID AND OD.ProductID = P.ProductID
GROUP BY ProductName
GO

if exists (select * from sysobjects where id = object_id('dbo.SalesByCategory') and sysstat & 0xf = 4)
	drop procedure dbo.SalesByCategory
GO
CREATE PROCEDURE SalesByCategory
    @CategoryName nvarchar(15), @OrdYear nvarchar(4) = '1998'
AS
IF @OrdYear != '1996' AND @OrdYear != '1997' AND @OrdYear != '1998' 
BEGIN
	SELECT @OrdYear = '1998'
END

SELECT ProductName,
	TotalPurchase=ROUND(SUM(CONVERT(decimal(14,2), OD.Quantity * (1-OD.Discount) * OD.UnitPrice)), 0)
FROM [Order Details] OD, Orders O, Products P, Categories C
WHERE OD.OrderID = O.OrderID 
	AND OD.ProductID = P.ProductID 
	AND P.CategoryID = C.CategoryID
	AND C.CategoryName = @CategoryName
	AND SUBSTRING(CONVERT(nvarchar(22), O.OrderDate, 111), 1, 4) = @OrdYear
GROUP BY ProductName
ORDER BY ProductName

"""

sql_code_parser = SqlCodeParser(
    source_directory="source_code/sql_server",
    source_file_glob_pattern="**/*.sql",
    chunk_limit=2,
    verbose=True,
)

print(sql_code_parser._extract_procedure_declaration_from_code('CustOrdersDetail', code))
# print(sql_code_parser._extract_procedure_declaration_from_code('SalesByCategory', code))


CREATE PROCEDURE CustOrdersDetail @OrderID int
AS
SELECT ProductName,
    UnitPrice=ROUND(Od.UnitPrice, 2),
    Quantity,
    Discount=CONVERT(int, Discount * 100), 
    ExtendedPrice=ROUND(CONVERT(money, Quantity * (1 - Discount) * Od.UnitPrice), 2)
FROM Products P, [Order Details] Od
WHERE Od.ProductID = P.ProductID and Od.OrderID = @OrderID
go


In [19]:
procedure_name = procedure_names[2]
sql_code = procedures[procedures['db_object_name'] == procedure_name]['sql_code'].values[0]
procedure_code = extract_procedure_declaration_from_code(procedure_name, sql_code)
# print(procedure_name)
# print(code)
print(procedure_code)

create procedure "Sales by Year" 
	@Beginning_Date DateTime, @Ending_Date DateTime AS
SELECT Orders.ShippedDate, Orders.OrderID, "Order Subtotals".Subtotal, DATENAME(yy,ShippedDate) AS Year
FROM Orders INNER JOIN "Order Subtotals" ON Orders.OrderID = "Order Subtotals".OrderID
WHERE Orders.ShippedDate Between @Beginning_Date And @Ending_Date
GO


In [20]:
def find_tables_manipulated_by_procedure(procedure_name, sql_code):
    chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, verbose=True)
    system_message_prompt = SystemMessagePromptTemplate.from_template("""
        Your are a SQL code parser.

        Find all the database tables that are manipulated by the {procedure_name} stored procedure.
        Find all the tables that are queried, inserted into, updated or deleted from and extract the 
        table name and database operation type (i.e. SELECT, INSERT, UPDATE, or DELETE).

        ## OUTPUT FORMAT ##
        json object array, containing the name of the table and the DML statement type in UPPERCASE text.
        Example:
        [{{ "table_name": "Order Details", "ddl_operation": "SELECT"}}]

        If there are no items, then return an empty json array.
    """)
    example1_prompt = HumanMessagePromptTemplate.from_template("""
    CREATE PROCEDURE CustOrdersDetail @OrderID int
    AS
    SELECT ProductName,
        UnitPrice=ROUND(Od.UnitPrice, 2),
        Quantity,
        Discount=CONVERT(int, Discount * 100), 
        ExtendedPrice=ROUND(CONVERT(money, Quantity * (1 - Discount) * Od.UnitPrice), 2)
    FROM Products P, [Order Details] Od
    WHERE Od.ProductID = P.ProductID and Od.OrderID = @OrderID
    go
    """)
    example1_response = AIMessagePromptTemplate.from_template('[{{ "table_name": "Products", "dml_operation": "SELECT"}}, {{ "table_name": "Order Details", "dml_operation": "SELECT"}}]')

    example2_prompt = HumanMessagePromptTemplate.from_template("""
    CREATE PROCEDURE CustOrdersOrders @CustomerID nchar(5)
    AS
    SELECT OrderID, 
        OrderDate,
        RequiredDate,
        ShippedDate
    FROM Orders
    WHERE CustomerID = @CustomerID
    ORDER BY OrderID
    GO
    """)
    example2_response = AIMessagePromptTemplate.from_template('[{{ "table_name": "Orders", "dml_operation": "SELECT"}}]')

    final_prompt = HumanMessagePromptTemplate.from_template("{sql_code_fragment}")

    chat_prompt = ChatPromptTemplate.from_messages([
        system_message_prompt, 
        example1_prompt, example1_response,
        example2_prompt, example2_response,
        final_prompt
    ])

    # get a chat completion from the formatted messages
    llm_response = chat(chat_prompt.format_prompt(procedure_name=procedure_name, sql_code_fragment=sql_code).to_messages())
    result = json.loads(llm_response.content)
    return result


In [21]:
procedure_name = 'SalesByCategory' #procedure_names[2]
sql_code = procedures[(procedures['ddl_operation'] == 'CREATE PROCEDURE') & (procedures['db_object_name'] == procedure_name)]['sql_code'].values[0]
procedure_code = extract_procedure_declaration_from_code(procedure_name, sql_code)
tables = find_tables_manipulated_by_procedure(procedure_name, procedure_code)
print("Procedure: ", procedure_name)
print("SQL code:\n------------------------\n", sql_code)
print(f"Extracted procedure code:\n------------------------\n{procedure_code}")
print("Tables manipulated by procedure:\n------------------------\n", tables)

Procedure:  SalesByCategory
SQL code:
------------------------
 GO

CREATE PROCEDURE CustOrdersDetail @OrderID int
AS
SELECT ProductName,
    UnitPrice=ROUND(Od.UnitPrice, 2),
    Quantity,
    Discount=CONVERT(int, Discount * 100), 
    ExtendedPrice=ROUND(CONVERT(money, Quantity * (1 - Discount) * Od.UnitPrice), 2)
FROM Products P, [Order Details] Od
WHERE Od.ProductID = P.ProductID and Od.OrderID = @OrderID
go


if exists (select * from sysobjects where id = object_id('dbo.CustOrdersOrders'))
	drop procedure dbo.CustOrdersOrders
GO

CREATE PROCEDURE CustOrdersOrders @CustomerID nchar(5)
AS
SELECT OrderID, 
	OrderDate,
	RequiredDate,
	ShippedDate
FROM Orders
WHERE CustomerID = @CustomerID
ORDER BY OrderID
GO


if exists (select * from sysobjects where id = object_id('dbo.CustOrderHist') and sysstat & 0xf = 4)
	drop procedure dbo.CustOrderHist
GO
CREATE PROCEDURE CustOrderHist @CustomerID nchar(5)
AS
SELECT ProductName, Total=SUM(Quantity)
FROM Products P, [Order Details] OD, Orders O

In [25]:
def group_tables_by_dml_operation(array_of_table_names_and_dml_operations):
    """
    Given an array of dictionaries, group the dictionaries by the dml_operation key
    and return a dictionary with the dml_operation as the key and the table_name as the value.

    Example:
    Input: [{'table_name': 'Orders', 'dml_operation': 'SELECT'}, {'table_name': 'Order Subtotals', 'dml_operation': 'SELECT'}]
    Output: {'SELECT': 'Orders, Order Subtotals'}
    """
    dict_by_operation = {}

    # Iterate over the array
    for item in array_of_table_names_and_dml_operations:
        # If the dml_operation is not in the dictionary, add it with the table_name as the value
        if item['dml_operation'] not in dict_by_operation:
            dict_by_operation[item['dml_operation']] = item['table_name']
        # If the dml_operation is already in the dictionary, append the table_name to the existing value
        else:
            dict_by_operation[item['ddl_operation']] += ', ' + item['table_name']
    return dict_by_operation

array = [{'table_name': 'Orders', 'dml_operation': 'SELECT'}, {'table_name': 'Order Subtotals', 'dml_operation': 'SELECT'}]
print(group_tables_by_dml_operation(array))


{'SELECT': 'Orders, Order Subtotals'}


In [35]:
procedure_names

['Ten Most Expensive Products',
 'Employee Sales by Country',
 'Sales by Year',
 'CustOrdersDetail',
 'CustOrdersOrders',
 'CustOrderHist',
 'SalesByCategory',
 'InsertProduct',
 'UpdateProduct',
 'DeleteProduct',
 'InsertOrder',
 'UpdateOrder',
 'DeleteOrder',
 'InsertOrderDetails',
 'UpdateOrderDetails',
 'DeleteOrderDetails',
 'ExecuteInsertProduct',
 'byroyalty',
 'reptq1',
 'reptq2',
 'reptq3']

In [36]:
# Create a new dataframe with table_name, dml_operation, and procedure_name columns
def create_dataframe_with_table_name_dml_operation_and_procedure_name(source_df, procedure_names):
    procedures_ds = source_df[source_df['ddl_operation'] == 'CREATE PROCEDURE']

    output_df = pd.DataFrame(columns=['table_name', 'dml_operation', 'procedure_name'])
    for procedure_name in procedure_names:
        sql_code = procedures_ds[procedures_ds['db_object_name'] == procedure_name]['sql_code'].values[0]
        procedure_code = extract_procedure_declaration_from_code(procedure_name, sql_code)
        
        tables = find_tables_manipulated_by_procedure(procedure_name, procedure_code)
        for table in tables:
            new_row = pd.DataFrame([{
                'table_name': table['table_name'], 
                'dml_operation': table['dml_operation'], 
                'procedure_name': procedure_name
            }])
            output_df = pd.concat([output_df, new_row], ignore_index=True)
    
    return output_df

final_df = create_dataframe_with_table_name_dml_operation_and_procedure_name(source_insights_dataframe, procedure_names)
final_df.head(30)


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Error in StdOutCallbackHandler.on_retry callback: 'StdOutCallbackHandler' object has no attribute 'on_retry'


,table_name,dml_operation,procedure_name
0,Products,SELECT,Ten Most Expensive Products
1,Employees,SELECT,Employee Sales by Country
2,Orders,SELECT,Employee Sales by Country
3,Order Subtotals,SELECT,Employee Sales by Country
4,Orders,SELECT,Sales by Year
5,Order Subtotals,SELECT,Sales by Year
6,Products,SELECT,CustOrdersDetail
7,Order Details,SELECT,CustOrdersDetail
8,Orders,SELECT,CustOrdersOrders
9,Products,SELECT,CustOrderHist


In [44]:
# Define a function to map 'dml_operation' to 'READ' or 'WRITE'
def map_sql_operation_to_read_write(operation):
    if operation == 'SELECT':
        return 'READ'
    elif operation in ['INSERT', 'UPDATE', 'DELETE']:
        return 'WRITE'
    else:
        return 'NONE' # In cases where there is neither a READ nor WRITE, such as calling a stored procedure.

# Apply the function to the 'dml_operation' column and assign the result to a new column
final_df['operation_type'] = final_df['dml_operation'].apply(map_sql_operation_to_read_write)
final_df.head()

,table_name,dml_operation,procedure_name,operation_type,combined_feature
0,Products,SELECT,Ten Most Expensive Products,READ,Products Ten Most Expensive Products READ
1,Employees,SELECT,Employee Sales by Country,READ,Employees Employee Sales by Country READ
2,Orders,SELECT,Employee Sales by Country,READ,Orders Employee Sales by Country READ
3,Order Subtotals,SELECT,Employee Sales by Country,READ,Order Subtotals Employee Sales by Country READ
4,Orders,SELECT,Sales by Year,READ,Orders Sales by Year READ


In [45]:
# Save the results to a CSV file
procedure_to_table_map_path="./results/tables_mapped_to_procedures.csv"
final_df.to_csv(procedure_to_table_map_path, index=False)

In [46]:
def cluster_procedures_by_table_and_operation(df, number_of_clusters=5):
    # Step 1: Define the features that you want to use to cluster the records
    # Combine 'table_name' and 'procedure_name' into a single feature
    df['combined_feature'] = df['table_name'] + ' ' + df['procedure_name'] + ' ' + df['operation_type']

    # Convert the 'combined_feature' column to a matrix of TF-IDF features
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['combined_feature'])

    # Step 2: Use a clustering algorithm to cluster the records based on these features
    # Let's use KMeans as an example
    kmeans = KMeans(n_clusters=number_of_clusters)
    kmeans.fit(X)

    # Assign the cluster labels to the records
    df['cluster_label'] = kmeans.labels_
    return df


cluster_df = cluster_procedures_by_table_and_operation(final_df, number_of_clusters=5)

# Step 3: Output the groups of records that are considered to be similar to each other
for label in cluster_df['cluster_label'].unique():
    print(f"Cluster {label}:")
    selected_procs = cluster_df[cluster_df['cluster_label'] == label]
    print(selected_procs[['procedure_name', 'operation_type', 'table_name', 'cluster_label']].sort_values(by=['procedure_name', 'operation_type', 'table_name']))



Cluster 2:
                 procedure_name operation_type table_name  cluster_label
9                 CustOrderHist           READ   Products              2
6              CustOrdersDetail           READ   Products              2
19                DeleteProduct          WRITE   Products              2
17                InsertProduct          WRITE   Products              2
15              SalesByCategory           READ   Products              2
0   Ten Most Expensive Products           READ   Products              2
18                UpdateProduct          WRITE   Products              2
Cluster 1:
              procedure_name operation_type       table_name  cluster_label
1  Employee Sales by Country           READ        Employees              1
3  Employee Sales by Country           READ  Order Subtotals              1
2  Employee Sales by Country           READ           Orders              1
5              Sales by Year           READ  Order Subtotals              1
4             

/Users/neil/.local/share/virtualenvs/src_insights-WRCfmHYa/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [47]:
def derive_service_name(group):
    # Prefer to name service according to which tables they write to, if possible
    group_write = group[group['operation_type'] == 'WRITE']

    if group_write.empty:
        if group.size > 0:
            most_common_table = group['table_name'].value_counts().idxmax()
            return most_common_table
        else:
            return 'Service With No Name'

    most_common_table = group_write['table_name'].value_counts().idxmax()
    return most_common_table


def add_derived_service_name_for_each_cluster_based_on_most_common_table_name(df):
    # Derive the service name for each cluster, pass the table_name and operation_type columns 
    # to the derive_service_name function.  Assign the service names to a new column called service_name.
    service_names = df.groupby('cluster_label')[['table_name', 'operation_type']].apply(derive_service_name).reset_index()
    service_names.columns = ['cluster_label', 'service_name']

    # Merge the service names into the original DataFrame
    df = pd.merge(cluster_df, service_names, on='cluster_label')

    return df


cluster_df = add_derived_service_name_for_each_cluster_based_on_most_common_table_name(cluster_df)
cluster_df.head(30)


,table_name,dml_operation,procedure_name,operation_type,combined_feature,cluster_label,service_name
0,Products,SELECT,Ten Most Expensive Products,READ,Products Ten Most Expensive Products READ,2,Products
1,Products,SELECT,CustOrdersDetail,READ,Products CustOrdersDetail READ,2,Products
2,Products,SELECT,CustOrderHist,READ,Products CustOrderHist READ,2,Products
3,Products,SELECT,SalesByCategory,READ,Products SalesByCategory READ,2,Products
4,Products,INSERT,InsertProduct,WRITE,Products InsertProduct WRITE,2,Products
5,Products,UPDATE,UpdateProduct,WRITE,Products UpdateProduct WRITE,2,Products
6,Products,DELETE,DeleteProduct,WRITE,Products DeleteProduct WRITE,2,Products
7,Employees,SELECT,Employee Sales by Country,READ,Employees Employee Sales by Country READ,1,Orders
8,Orders,SELECT,Employee Sales by Country,READ,Orders Employee Sales by Country READ,1,Orders
9,Order Subtotals,SELECT,Employee Sales by Country,READ,Order Subtotals Employee Sales by Country READ,1,Orders


In [48]:
def create_service_diagram(service_name, procedures, read_tables, write_tables):
  """
  Given a service name, procedures, read tables, and write tables, create a PlantUML diagram.
  """    
  diagram_template_text = """
  @startuml "{{ service_name }}"

  class {{ service_name }} <<domain service>> {
    {%- for procedure in procedures %}
    + {{ procedure }}() <<api>>
    {%- endfor %}
  }

  package "{{ service_name }}_PROCS" {
    {% for procedure in procedures %}
    class {{ procedure }} <<proc>> {
    }
    {% endfor %}
  }

  package "{{ service_name }}_READS" {
    {% for table in read_tables %}
    class {{ table }} <<table>> {
    }
    {% endfor %}
  }

  package "{{ service_name }}_WRITES" {
    {% for table in write_tables %}
    class {{ table }} <<table>> {
    }
    {% endfor %}
  }

  {{ service_name }} --> "{{ service_name }}_PROCS" : calls
  {{ service_name }}_PROCS --> "{{ service_name }}_READS" : reads
  {{ service_name }}_PROCS --> "{{ service_name }}_WRITES" : writes

  @enduml
  """

  template = Template(diagram_template_text)
  rendered_text = template.render(service_name=service_name, procedures=procedures, read_tables=read_tables, write_tables=write_tables)
  # print(rendered_text)

  diagram_path = f"./results/{service_name}.puml"
  with open(diagram_path, 'w') as file:
        file.write(rendered_text)

  cmd = f'plantuml {diagram_path}'
  subprocess.run(cmd, shell=True, check=True)




In [49]:
def replace_spaces_with_underscores_in_names(df):
    """
    Replace spaces with underscores in the service_name, table_name and procedure_name columns
    """
    for col in ['service_name', 'procedure_name', 'table_name']:
        df[col] = df[col].str.replace(' ', '_')
    return df

cluster_df = replace_spaces_with_underscores_in_names(cluster_df)

def create_service_diagrams(df):
    """
    Create a PlantUML diagram for each service.

    The diagram will show the procedures, read tables, and write tables for each service.
    """
    for service_name in df['service_name'].unique():
        procedures = df[df['service_name'] == service_name]['procedure_name'].unique()
        read_tables = df[(df['service_name'] == service_name) & (df['operation_type'] == 'READ')]['table_name'].unique()
        write_tables = df[(df['service_name'] == service_name) & (df['operation_type'] == 'WRITE')]['table_name'].unique()
        create_service_diagram(service_name, procedures, read_tables, write_tables)
  
create_service_diagrams(cluster_df)
